<a href="https://colab.research.google.com/github/a-v-kolos/Allen_dataset_neuromatch_2025/blob/sebastien_bullich/SebTryingStuff_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive

# ---------------------------------------------------------------
# Mount Google Drive and define output path
# ---------------------------------------------------------------
drive.mount("/content/drive", force_remount=True)
OUTPUT_DIR = "/content/drive/MyDrive/pca_drift_results5"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ---------------------------------------------------------------
# Parameters
# ---------------------------------------------------------------
SESSION_COL   = "ophys_session_id"
EARLY_FRAC    = 0.25
LATE_FRAC     = 0.25
MAX_PC        = 20
INCLUDED_IMAGES = ["im000", "im106", "im075", "im073", "im045", "im054", "im031", "im035"]

# Set plot style
sns.set_theme(style="whitegrid", context="talk", font_scale=1.0, palette="colorblind")

# Results container
rows = []

# Extracting data
file_path = "/content/drive/MyDrive/allen_visual_behavior_2p_change_detection_familiar_novel_image_sets.parquet"
data = pd.read_parquet(file_path)
sst_data_2 = data[
    (data.cre_line           == "Sst-IRES-Cre") &
    (data.omitted            == False) &
    (data.exposure_level     == "familiar") &
    (data.targeted_structure == "VISp") &
    (data.session_type       == "OPHYS_3_images_B")
]

# Loop over all sessions and images
for sess in sst_data_2[SESSION_COL].unique():
    df_sess = sst_data_2[sst_data_2[SESSION_COL] == sess].copy()

    # Create a subfolder for this mouse/session
    #mouse_plot_dir = os.path.join(OUTPUT_DIR, "plots", f"session_{sess}")
    #os.makedirs(mouse_plot_dir, exist_ok=True)

    # Get animal ID for this session (assumes 1 per session)
    mouse_id = df_sess["mouse_id"].iloc[0] if "mouse_id" in df_sess.columns else "unknown"

    # Create subfolder using both mouse and session
    mouse_plot_dir = os.path.join(OUTPUT_DIR, "plots", f"mouse_{mouse_id}_session_{sess}")
    os.makedirs(mouse_plot_dir, exist_ok=True)

    for img in df_sess["image_name"].unique():
        if img not in INCLUDED_IMAGES:
            continue

        df_img = df_sess[df_sess["image_name"] == img].copy()
        if len(df_img) < 2:
            continue

        # Δ response and block labels
        df_img["delta_response"] = df_img["mean_response"] - df_img["baseline_response"]
        df_img = df_img.sort_values("stimulus_presentations_id").reset_index(drop=True)
        N = len(df_img)
        early_cut  = max(1, int(np.floor(EARLY_FRAC * N)))
        late_start = N - max(1, int(np.ceil(LATE_FRAC * N)))
        df_img["block"] = np.where(
            df_img.index < early_cut, "early",
            np.where(df_img.index >= late_start, "late", "middle")
        )

     # Population matrix
        M_df = (
            df_img
            .pivot_table(index="cell_specimen_id",
                        columns="stimulus_presentations_id",
                        values="delta_response",
                        aggfunc="mean")
            .dropna(axis=0, how="all")
            .dropna(axis=1, how="all")
        )

        # Now remove remaining rows/columns with any NaNs (safer than fillna(0.0))
        M_df = M_df.dropna(axis=0, how="any").dropna(axis=1, how="any")

        M = M_df.to_numpy()
        if M.shape[0] < 2 or M.shape[1] < 2:
            continue

        trials = M.shape[1]
        cells = M.shape[0]

        # Block masks
        trial_blocks = (
            df_img[["stimulus_presentations_id", "block"]]
            .drop_duplicates("stimulus_presentations_id")
            .set_index("stimulus_presentations_id")
            .reindex(M_df.columns)["block"]
            .fillna("unknown")
            .to_numpy()
        )
        is_early = trial_blocks == "early"
        is_late  = trial_blocks == "late"

        if not is_early.any() or not is_late.any():
            continue

        # Drift calculation
        v_e = np.mean(M[:, is_early], axis=1)
        v_l = np.mean(M[:, is_late], axis=1)
        if np.isnan(v_e).all() or np.isnan(v_l).all():
            continue

        drift = v_l - v_e
        mag = np.linalg.norm(np.nan_to_num(drift))
        denom = np.linalg.norm(np.nan_to_num(v_e)) * np.linalg.norm(np.nan_to_num(v_l))
        cosang = np.clip((np.nan_to_num(v_e) @ np.nan_to_num(v_l)) / denom if denom else 1, -1, 1)
        angle = np.degrees(np.arccos(cosang))

        rows.append({
            "ophys_session_id": sess,
            "image_name": img,
            "drift_magnitude": mag,
            "drift_angle": angle
        })

        # PCA and plot
        n_comp = min(MAX_PC, trials, cells)
        pca = PCA(n_components=n_comp).fit(M.T)
        scores = pca.transform(M.T)

        if n_comp >= 2:
            fig, ax = plt.subplots(figsize=(5, 5))
            ax.scatter(scores[is_early, 0], scores[is_early, 1],
                       label="early", alpha=0.7, s=40, edgecolor='k')
            ax.scatter(scores[is_late, 0], scores[is_late, 1],
                       label="late", alpha=0.7, s=40, edgecolor='k', marker='^')

            ec = scores[is_early, :2].mean(axis=0)
            lc = scores[is_late, :2].mean(axis=0)
            ax.scatter(*ec, s=100, color='red', marker='o', edgecolor='k')
            ax.scatter(*lc, s=100, color='blue', marker='^', edgecolor='k')
            ax.annotate("", xy=lc, xytext=ec,
                        arrowprops=dict(arrowstyle="->", lw=2, color="gray"))

            ax.set_xlabel("PC1")
            ax.set_ylabel("PC2")
            ax.set_title(f"Session {sess} — Image {img}")
            ax.legend()
            fig.tight_layout()
            fig.savefig(os.path.join(mouse_plot_dir, f"image_{img}.png"))
            plt.close(fig)

# Save drift table
df_out = pd.DataFrame(rows)
df_out.to_csv(os.path.join(OUTPUT_DIR, "drift_metrics_filtered.csv"), index=False)

# Confirm results
print("✅ Done!")
print(f"Saved {len(df_out)} drift metric rows for filtered images")
print("Results saved to:", OUTPUT_DIR)


Mounted at /content/drive
✅ Done!
Saved 64 drift metric rows for filtered images
Results saved to: /content/drive/MyDrive/pca_drift_results5
